# MODUPE OYEDIRAN
##modupeolayinkaoyediran@gmail.com

Movielens Data Analytics
--------------------------
For this notebook, we will be using a MovieLens sample dataset. The data includes 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users and can be found in https://grouplens.org/datasets/movielens/latest/

We load the data we just downloaded from the MovieLens website. The files were uploaded to an S3 bucket and loaded from there.

In [0]:
links = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/FileStore/shared_uploads/dupe_oyediran@yahoo.com/links.csv")
movies = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/FileStore/shared_uploads/dupe_oyediran@yahoo.com/movies.csv")
tags = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/FileStore/shared_uploads/dupe_oyediran@yahoo.com/tags.csv")
ratings = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/FileStore/shared_uploads/dupe_oyediran@yahoo.com/ratings.csv")

Once the data is loaded, we take a look at them by using the show action is spark.

In [0]:
links.show()

+-------+------+------+
movieId|imdbId|tmdbId|
+-------+------+------+
 1|114709| 862|
 2|113497| 8844|
 3|113228| 15602|
 4|114885| 31357|
 5|113041| 11862|
 6|113277| 949|
 7|114319| 11860|
 8|112302| 45325|
 9|114576| 9091|
 10|113189| 710|
 11|112346| 9087|
 12|112896| 12110|
 13|112453| 21032|
 14|113987| 10858|
 15|112760| 1408|
 16|112641| 524|
 17|114388| 4584|
 18|113101| 5|
 19|112281| 9273|
 20|113845| 11517|
+-------+------+------+
only showing top 20 rows

In [0]:
movies.show()

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
 6| Heat (1995)|Action|Crime|Thri...|
 7| Sabrina (1995)| Comedy|Romance|
 8| Tom and Huck (1995)| Adventure|Children|
 9| Sudden Death (1995)| Action|
 10| GoldenEye (1995)|Action|Adventure|...|
 11|American Presiden...|Comedy|Drama|Romance|
 12|Dracula: Dead and...| Comedy|Horror|
 13| Balto (1995)|Adventure|Animati...|
 14| Nixon (1995)| Drama|
 15|Cutthroat Island ...|Action|Adventure|...|
 16| Casino (1995)| Crime|Drama|
 17|Sense and Sensibi...| Drama|Romance|
 18| Four Rooms (1995)| Comedy|
 19|Ace Ventura: When...| Comedy|
 20| Money Train (1995)|Action|Comedy|Cri...|
+-------+--------------------+--------------------+
only showing top 20 rows

In [0]:
tags.show()

+------+-------+-----------------+----------+
userId|movieId| tag| timestamp|
+------+-------+-----------------+----------+
 2| 60756| funny|1445714994|
 2| 60756| Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
 2| 89774| Tom Hardy|1445715205|
 2| 106782| drugs|1445715054|
 2| 106782|Leonardo DiCaprio|1445715051|
 2| 106782| Martin Scorsese|1445715056|
 7| 48516| way too long|1169687325|
 18| 431| Al Pacino|1462138765|
 18| 431| gangster|1462138749|
 18| 431| mafia|1462138755|
 18| 1221| Al Pacino|1461699306|
 18| 1221| Mafia|1461699303|
 18| 5995| holocaust|1455735472|
 18| 5995| true story|1455735479|
 18| 44665| twist ending|1456948283|
 18| 52604| Anthony Hopkins|1457650696|
 18| 52604| courtroom drama|1457650711|
+------+-------+-----------------+----------+
only showing top 20 rows

In [0]:
ratings.show()

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

As we cans see in the table previews, the year of the movie is embeded in the movie title. In order to extract this information, I created a User Defined Function(UDF) that takes the year when available, otherwise it returns null.

In [0]:
from pyspark.sql.functions import udf
import pyspark.sql.functions as F

def get_year(title):
  try:
    return(int(title[-5:-1]))
  except:
    return(None)

get_year_udf = udf(get_year)

In [0]:
movies = movies.withColumn("year", get_year_udf(movies.title))
movies.show()

+-------+--------------------+--------------------+----+
movieId| title| genres|year|
+-------+--------------------+--------------------+----+
 1| Toy Story (1995)|Adventure|Animati...|1995|
 2| Jumanji (1995)|Adventure|Childre...|1995|
 3|Grumpier Old Men ...| Comedy|Romance|1995|
 4|Waiting to Exhale...|Comedy|Drama|Romance|1995|
 5|Father of the Bri...| Comedy|1995|
 6| Heat (1995)|Action|Crime|Thri...|1995|
 7| Sabrina (1995)| Comedy|Romance|1995|
 8| Tom and Huck (1995)| Adventure|Children|1995|
 9| Sudden Death (1995)| Action|1995|
 10| GoldenEye (1995)|Action|Adventure|...|1995|
 11|American Presiden...|Comedy|Drama|Romance|1995|
 12|Dracula: Dead and...| Comedy|Horror|1995|
 13| Balto (1995)|Adventure|Animati...|1995|
 14| Nixon (1995)| Drama|1995|
 15|Cutthroat Island ...|Action|Adventure|...|1995|
 16| Casino (1995)| Crime|Drama|1995|
 17|Sense and Sensibi...| Drama|Romance|1995|
 18| Four Rooms (1995)| Comedy|1995|
 19|Ace Ventura: When...| Comedy|1995|
 20| Money Train (1995)|Action|Comedy|Cri...|1995|
+-------+--------------------+--------------------+----+
only showing top 20 rows

# Insights

As part of the analysis of this dataset, it would be useful to have the average rating for each movie. In the following cell of code, I aggregate over the ratings table to get the average rating for each movie ID.

In [0]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import bround
from pyspark.sql.functions import mean

ratings_agg = ratings.groupBy("movieId").agg(mean("rating").alias("avg_rating"))
ratings_agg = ratings_agg.withColumn("average_rating", ratings_agg.avg_rating.cast(FloatType())).drop("avg_rating").withColumnRenamed("average_rating", "avg_rating")
ratings_agg = ratings_agg.select("movieId",bround("avg_rating",2).alias("avg_rating"))
ratings_agg.show()

+-------+----------+
movieId|avg_rating|
+-------+----------+
 1580| 3.49|
 2366| 3.64|
 3175| 3.58|
 1088| 3.37|
 32460| 4.25|
 44022| 3.22|
 96488| 4.25|
 1238| 4.06|
 1342| 2.5|
 1591| 2.63|
 1645| 3.41|
 4519| 3.33|
 2142| 2.7|
 471| 3.55|
 3997| 1.83|
 833| 2.0|
 3918| 3.28|
 7982| 3.25|
 1959| 3.67|
 68135| 3.55|
+-------+----------+
only showing top 20 rows

In [0]:
joined_movies = movies.join(ratings_agg,"movieId")
joined_movies.show()

+-------+--------------------+--------------------+----+----------+
movieId| title| genres|year|avg_rating|
+-------+--------------------+--------------------+----+----------+
 1580|Men in Black (a.k...|Action|Comedy|Sci-Fi|1997| 3.49|
 2366| King Kong (1933)|Action|Adventure|...|1933| 3.64|
 3175| Galaxy Quest (1999)|Adventure|Comedy|...|1999| 3.58|
 1088|Dirty Dancing (1987)|Drama|Musical|Rom...|1987| 3.37|
 32460|Knockin' on Heave...|Action|Comedy|Cri...|1997| 4.25|
 44022|Ice Age 2: The Me...|Adventure|Animati...|2006| 3.22|
 96488|Searching for Sug...| Documentary|2012| 4.25|
 1238| Local Hero (1983)| Comedy|1983| 4.06|
 1342| Candyman (1992)| Horror|Thriller|1992| 2.5|
 1591| Spawn (1997)|Action|Adventure|...|1997| 2.63|
 1645|The Devil's Advoc...|Drama|Mystery|Thr...|1997| 3.41|
 4519|Land Before Time,...|Adventure|Animati...|1988| 3.33|
 2142|American Tail: Fi...|Adventure|Animati...|1991| 2.7|
 471|Hudsucker Proxy, ...| Comedy|1994| 3.55|
 3997|Dungeons & Dragon...|Action|Adventure|...|2000| 1.83|
 833|High School High ...| Comedy|1996| 2.0|
 3918|Hellbound: Hellra...| Horror|1988| 3.28|
 7982|Tale of Two Siste...|Drama|Horror|Myst...|2003| 3.25|
 1959|Out of Africa (1985)| Drama|Romance|1985| 3.67|
 68135| 17 Again (2009)| Comedy|Drama|2009| 3.55|
+-------+--------------------+--------------------+----+----------+
only showing top 20 rows

Here we evaluate the average rating by year to identify if there is a trend in the ratings either to decrease or increase over the years. Visually, it is not possible to appreaciate such trend, but it was possible to identify some outlayer values in the year column.

To achieve this, it was necessary to join the aggregated ratings table with movies table that includes the year as a column.

In [0]:
joined_movies.select("year",'avg_rating').groupBy("year").mean().orderBy("year").display()

year,avg(avg_rating)
null,3.3958333333333335
1902,3.5
1903,2.5
1908,4.0
1915,2.0
1916,3.5625
1917,4.5
1919,2.0
1920,3.6799999475479126
1921,4.099999904632568


There does not seem to be any appreciable difference in movie ratings over the years considered.

As mentioned before, the year column contains some outlayers and null values. Thus, here I aggregate the data counting the number of movies by year. By doing this, we can see that there are some movies with years from the early 1900s.

In [0]:
joined_movies.select("year",'avg_rating').groupBy("year").count().orderBy("year").display()

year,count
null,24
1902,1
1903,1
1908,1
1915,1
1916,4
1917,1
1919,1
1920,2
1921,1


We can see that there is an exponential increase in the qauntity of movie produced over the years.

We want to see how each movie genre accounts for the quantity of movies reviewed.

In [0]:
from pyspark.sql.functions import split,explode

exploded_movies = movies.withColumn("genres", explode(split("genres","[|]")))
exploded_movies.groupBy("genres").count().display()

genres,count
Crime,1199
Romance,1596
Thriller,1894
Adventure,1263
Drama,4361
War,382
Documentary,440
Fantasy,779
Mystery,573
Musical,334


We can see that the Drama and Comedy genre are the most reviewed genre about doubling any of the other genres. This perhaps indicates that they are most liked by the public.

Something noteworthy to analyze is to check wether the voters have a bias on a particular genre. This could be tested by taking the average rating by genre and see whether the rating distribution is uniform, that is, that there is no systematic bias towards a particular genre.

In [0]:
rated_genres = exploded_movies.join(ratings,"movieId").select("genres","rating")
rated_genres.groupBy("genres").mean().display()

genres,avg(rating)
Crime,3.658293867274144
Romance,3.5065107040388437
Thriller,3.4937055799183425
Adventure,3.5086089151939075
Drama,3.6561844113718758
War,3.8082938876312
Documentary,3.797785069729286
Fantasy,3.4910005070136894
Mystery,3.632460255407871
Musical,3.5636781053649105


By doing a visual analysis, it seems that the people rating the movies has no bias towards a particular genre. The distribution looks quite uniform, even though the sample is small.

Let's now proceed to see which movies are the ones with the best score and those ones with the worst score. For this purpose, I will take the top 10 elements from the joined table that includes the movie title and the average rating.

In [0]:
from pyspark.sql.functions import desc
joined_movies.select("title","avg_rating").orderBy(desc("avg_rating")).head(10)

Out[119]: [Row(title='Empties (2007)', avg_rating=5.0),
 Row(title='Zeitgeist: Moving Forward (2011)', avg_rating=5.0),
 Row(title='Who Killed Chea Vichea? (2010)', avg_rating=5.0),
 Row(title='One I Love, The (2014)', avg_rating=5.0),
 Row(title='Martin Lawrence Live: Runteldat (2002)', avg_rating=5.0),
 Row(title='What Happened Was... (1994)', avg_rating=5.0),
 Row(title='Lamerica (1994)', avg_rating=5.0),
 Row(title='The Editor (2015)', avg_rating=5.0),
 Row(title='Raise Your Voice (2004)', avg_rating=5.0),
 Row(title='Adventures Of Sherlock Holmes And Dr. Watson: The Twentieth Century Approaches (1986)', avg_rating=5.0)]

Now I take the 10 movies with the lowest average rating to see the worst movies in our dataset.

In [0]:
joined_movies.select("title","avg_rating").orderBy("avg_rating").head(10)

Out[120]: [Row(title='In the Name of the King: A Dungeon Siege Tale (2008)', avg_rating=0.5),
 Row(title='Journey 2: The Mysterious Island (2012)', avg_rating=0.5),
 Row(title="Don't Look Now (1973)", avg_rating=0.5),
 Row(title='Derailed (2002)', avg_rating=0.5),
 Row(title='Son of God (2014)', avg_rating=0.5),
 Row(title='Sorrow (2015)', avg_rating=0.5),
 Row(title='Baxter (1989)', avg_rating=0.5),
 Row(title='Daddy Day Camp (2007)', avg_rating=0.5),
 Row(title='Christmas Carol, A (1977)', avg_rating=0.5),
 Row(title='Born to Be Wild (1995)', avg_rating=0.5)]

finally I save the dataframe to an S3 bucket in parquet format for storage to a AWS Redshift datawarehouse for later anayltics

In [0]:
joined_movies.write.format("parquet").mode(SaveMode.Overwrite).option("path","s3a://filestoragedatabricks/Spark-essentials-data/bands2.parquet").save()